In [24]:
#!unzip Test.zip

In [25]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.transform import resize 


In [26]:
suffix = '.pgm'
kaggle_files = os.listdir('Test')
pgm_kaggle_files = [filename for filename in kaggle_files if filename.endswith(suffix)]

In [27]:
len(pgm_kaggle_files)

8000

In [28]:
IMG_SIZE = (64, 64)  # Tamaño al que se redimensionarán las imágenes

kaggle_patches = []
for filename in tqdm(pgm_kaggle_files):
  path = 'Test/' + filename
  with open(path, 'rb') as pgmf:
      image = plt.imread(pgmf)
      img_resized = resize(image, IMG_SIZE, anti_aliasing=True)
      kaggle_patches.append(img_resized.flatten())

100%|██████████| 8000/8000 [00:02<00:00, 3709.67it/s]


In [29]:
pgm_kaggle_files_id = []

for filename in tqdm(pgm_kaggle_files):
  pgm_kaggle_files_id.append(filename[5:-4])

100%|██████████| 8000/8000 [00:00<00:00, 4030078.31it/s]


In [30]:
import joblib

In [31]:
# Aquí debe utilizar su modelo
model = joblib.load('models/knn_model.joblib')

In [32]:
from skimage import feature

# Aquí debe procesar las imagenes de pgm_kaggle_files
X_kag = []

suffix = '.pgm'
dir = os.listdir('Test/')
pgm_files = [filename for filename in dir if filename.endswith(suffix)]

for filename in tqdm(pgm_files):
    path = 'Test/' + filename
    with open (path, 'rb') as pgmf:
        image = plt.imread(pgmf)
        img_resized = resize(image, IMG_SIZE, anti_aliasing=True)
        X_kag.append(img_resized.flatten())
     
X = np.array(X_kag)

100%|██████████| 8000/8000 [00:01<00:00, 7280.10it/s]


In [33]:
X

array([[0.47843137, 0.47843137, 0.47058824, ..., 0.1372549 , 0.13333333,
        0.13333333],
       [0.15686275, 0.15294118, 0.15294118, ..., 0.17254902, 0.16862745,
        0.16470588],
       [0.74901961, 0.75686275, 0.77254902, ..., 0.58823529, 0.50980392,
        0.41176471],
       ...,
       [0.4       , 0.4       , 0.4       , ..., 0.63529412, 0.63137255,
        0.62745098],
       [0.77647059, 0.78431373, 0.78431373, ..., 0.63921569, 0.63529412,
        0.63529412],
       [0.87058824, 0.85490196, 0.82352941, ..., 0.63529412, 0.40392157,
        0.34117647]])

In [34]:
from sklearn.decomposition import PCA 

pca = PCA(n_components=200)
X_pca_kag = pca.fit_transform(X)

# Predicción
y_kag = model.predict(X_pca_kag)

In [35]:
# Predicción + ID
y_kag_dic = {pgm_kaggle_files_id[i]: y_kag[i] for i in range(len(pgm_kaggle_files_id))}
kaggle_hat = pd.DataFrame(list(y_kag_dic.items()), columns=['id', 'target_feature'])
kaggle_hat['id'] = kaggle_hat['id'].astype(int)
kaggle_hat['target_feature'] = kaggle_hat['target_feature'].astype(int)
kaggle_hat.sort_values(by='id', inplace=True)

In [36]:
kaggle_hat.head()

,id,target_feature
2069,0,1
2527,1,1
3030,2,1
2576,3,1
4060,4,0


In [37]:
from datetime import datetime

fecha_hora = datetime.now().strftime("%Y-%m-%d %H:%M")

print(fecha_hora)

2025-06-13 19:24


In [38]:
# Guardamos en un .csv para subir a kaggle
submission_name = f'CSV_samples/knn_{fecha_hora}'
kaggle_hat.to_csv(submission_name + '.csv', index=False)